In [1]:
!pip install neat-python

     |████████████████████████████████| 44 kB 1.7 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
from __future__ import print_function
import os
import pandas as pd
import numpy as np
import sys 
import os
import math
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from datetime import datetime
from IPython.display import clear_output

In [4]:
import sys
import neat
sys.path.insert(0,'/content/drive/My Drive/2. Adelaide Uni/Research Project/Evolutionary Models/Imports')
from UpperBoundCalculator import UpperBoundCalculator
from StatisticsReporter import StatisticsReporter
from Reporting import StdOutReporter
from Reporting import ReporterSet
from Reporting import BaseReporter



In [5]:
# Uploading the saved model state
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

def create_mount_pydrive():
    # 1. Authenticate and create the PyDrive client.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    u_drive = GoogleDrive(gauth)
    return u_drive


In [6]:
upload_drive = create_mount_pydrive()

In [7]:
"""Implements the core evolution algorithm."""
from neat.math_util import mean
from neat.six_util import iteritems, itervalues


class CompleteExtinctionException(Exception):
    pass

class Population(object):
    """
    This class implements the core evolution algorithm:
        1. Evaluate fitness of all genomes.
        2. Check to see if the termination criterion is satisfied; exit if it is.
        3. Generate the next generation from the current population.
        4. Partition the new generation into species based on genetic similarity.
        5. Go to 1.
    """

    def __init__(self, config, initial_state=None):
        self.reporters = ReporterSet()
        self.config = config
        stagnation = config.stagnation_type(config.stagnation_config, self.reporters)
        self.reproduction = config.reproduction_type(config.reproduction_config,
                                                     self.reporters,
                                                     stagnation)
        if config.fitness_criterion == 'max':
            self.fitness_criterion = max
        elif config.fitness_criterion == 'min':
            self.fitness_criterion = min
        elif config.fitness_criterion == 'mean':
            self.fitness_criterion = mean
        elif not config.no_fitness_termination:
            raise RuntimeError(
                "Unexpected fitness_criterion: {0!r}".format(config.fitness_criterion))

        if initial_state is None:
            # Create a population from scratch, then partition into species.
            self.population = self.reproduction.create_new(config.genome_type,
                                                           config.genome_config,
                                                           config.pop_size)
            self.species = config.species_set_type(config.species_set_config, self.reporters)
            self.generation = 0
            self.species.speciate(config, self.population, self.generation)
        else:
            self.population, self.species, self.generation = initial_state

        self.best_genome = None

    def add_reporter(self, reporter):
        self.reporters.add(reporter)

    def remove_reporter(self, reporter):
        self.reporters.remove(reporter)

    def run(self, fitness_function, n=None):
        """
        Runs NEAT's genetic algorithm for at most n generations.  If n
        is None, run until solution is found or extinction occurs.

        The user-provided fitness_function must take only two arguments:
            1. The population as a list of (genome id, genome) tuples.
            2. The current configuration object.

        The return value of the fitness function is ignored, but it must assign
        a Python float to the `fitness` member of each genome.

        The fitness function is free to maintain external state, perform
        evaluations in parallel, etc.

        It is assumed that fitness_function does not modify the list of genomes,
        the genomes themselves (apart from updating the fitness member),
        or the configuration object.
        """

        if self.config.no_fitness_termination and (n is None):
            raise RuntimeError("Cannot have no generational limit with no fitness termination")

        k = 0
        while n is None or k < n:
            k += 1

            self.reporters.start_generation(self.generation)

            # Evaluate all genomes using the user-provided function.
            fitness_function(list(iteritems(self.population)), self.config)

            # Gather and report statistics.
            best = None
            for g in itervalues(self.population):
                if best is None or g.fitness > best.fitness:
                    best = g
            self.reporters.post_evaluate(self.config, self.population, self.species, best)

            # Track the best genome ever seen.
            if self.best_genome is None or best.fitness > self.best_genome.fitness:
                self.best_genome = best

            if not self.config.no_fitness_termination:
                # End if the fitness threshold is reached.
                fv = self.fitness_criterion(g.fitness for g in itervalues(self.population))
                if fv >= self.config.fitness_threshold:
                    self.reporters.found_solution(self.config, self.generation, best)
                    break

            # Create the next generation from the current generation.
            self.population = self.reproduction.reproduce(self.config, self.species,
                                                          self.config.pop_size, self.generation)

            # Check for complete extinction.
            if not self.species.species:
                self.reporters.complete_extinction()

                # If requested by the user, create a completely new population,
                # otherwise raise an exception.
                if self.config.reset_on_extinction:
                    self.population = self.reproduction.create_new(self.config.genome_type,
                                                                   self.config.genome_config,
                                                                   self.config.pop_size)
                else:
                    raise CompleteExtinctionException()

            # Divide the new population into species.
            self.species.speciate(self.config, self.population, self.generation)
            upload_drive = create_mount_pydrive()

            self.reporters.end_generation(self.config, self.population, self.species, upload_drive=upload_drive)

            self.generation += 1

        if self.config.no_fitness_termination:
            self.reporters.found_solution(self.config, self.generation, self.best_genome)

        return self.best_genome

In [8]:
import gzip
import random
import time

try:
    import cPickle as pickle  # pylint: disable=import-error
except ImportError:
    import pickle  # pylint: disable=import-error
class Checkpointer(BaseReporter):
    """
    A reporter class that performs checkpointing using `pickle`
    to save and restore populations (and other aspects of the simulation state).
    """

    def __init__(self, generation_interval=100, time_interval_seconds=300,
                 filename_prefix='neat-model2-checkpoint-'):
        """
        Saves the current state (at the end of a generation) every ``generation_interval`` generations or
        ``time_interval_seconds``, whichever happens first.

        :param generation_interval: If not None, maximum number of generations between save intervals
        :type generation_interval: int or None
        :param time_interval_seconds: If not None, maximum number of seconds between checkpoint attempts
        :type time_interval_seconds: float or None
        :param str filename_prefix: Prefix for the filename (the end will be the generation number)
        """
        self.generation_interval = generation_interval
        self.time_interval_seconds = time_interval_seconds
        self.filename_prefix = filename_prefix

        self.current_generation = None
        self.last_generation_checkpoint = -1
        self.last_time_checkpoint = time.time()

    def start_generation(self, generation):
        self.current_generation = generation

    def end_generation(self, config, population, species_set, upload_drive=None):
        checkpoint_due = False

        if self.time_interval_seconds is not None:
            dt = time.time() - self.last_time_checkpoint
            if dt >= self.time_interval_seconds:
                checkpoint_due = True

        if (checkpoint_due is False) and (self.generation_interval is not None):
            dg = self.current_generation - self.last_generation_checkpoint
            if dg >= self.generation_interval:
                checkpoint_due = True

        if (checkpoint_due) and (upload_drive is not None):
            self.save_checkpoint(config, population, species_set, self.current_generation, upload_drive)
            self.last_generation_checkpoint = self.current_generation
            self.last_time_checkpoint = time.time()

    def save_checkpoint(self, config, population, species_set, generation, upload_drive):
        """ Save the current simulation state. """
        filename = '{0}{1}'.format(self.filename_prefix, generation)
        print("Saving checkpoint to {0}".format(filename))

        with gzip.open(filename, 'w', compresslevel=5) as f:
            data = (generation, config, population, species_set, random.getstate())
            pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

        folder_id = '1NJ8abLpYebjVZscfuXmCdbrar8Gp67i2'
        model_checkpoint = upload_drive.CreateFile({'parents':[{u'id': folder_id}]})
        model_checkpoint.SetContentFile(filename)
        model_checkpoint.Upload()


    @staticmethod
    def restore_checkpoint(filename):
        """Resumes the simulation from a previous saved point."""
        with gzip.open(filename) as f:
            generation, config, population, species_set, rndstate = pickle.load(f)
            random.setstate(rndstate)
            return Population(config, (population, species_set, generation))

In [9]:
#  Loading or starting from scratch
option_not_set=True
reload_mode = False
reload_filename = ''
stats_filepath = ''
init_lr = 1e-3
while (option_not_set):
    # clear_output(wait=True)
    print("Please enter number to choose model reload mode:\n")
    print("1. Reload existing model")
    print("2. Start from scratch\n")
    option = input("option: ")
    if (option=='1'):
        option_not_set = False
        reload_mode = True
        init_lr = 1e-5
        print("\nPlease enter filename to reload model from:")
        reload_filename = input("filename: ")
        print(f'Loading population from {reload_filename} ...')
        # Loading trained model
        drive.mount('/content/gdrive', force_remount=True)
        GDRIVE_DIR = "gdrive/My Drive/2. Adelaide Uni/Research Project/Evolutionary Models/Checkpoints"
        restored_population = Checkpointer.restore_checkpoint(os.path.join(GDRIVE_DIR, reload_filename))

        print("Please enter filename for model stats")        
        stats_filename = input("filename for model stats: ")
        stats_filepath = os.path.join(GDRIVE_DIR, "Stats/"+stats_filename)    
        stats_reporter = StatisticsReporter(generation_interval=20, filename_prefix='neat-model2-stats-checkpoint-')
        restore_checkpoint_result = stats_reporter.restore_checkpoint(stats_filepath)
        if restore_checkpoint_result==0:
            break
        else:
            print(restore_checkpoint_result)
    elif (option=='2'):
        option_not_set = False
        print("Starting model from scratch!\n")
        init_lr = 1e-4
        break
    else:
        print("Invalid input, try again!\n")
        continue

# neat-model2-stats-checkpoint-299
# neat-model2-checkpoint-299

Please enter number to choose model reload mode:

1. Reload existing model
2. Start from scratch

option: 2
Starting model from scratch!



In [10]:
if (reload_mode):
    for n, genome in enumerate(stats_reporter.most_fit_genomes):
        print(f'Generation {n}:{genome.fitness}')

In [11]:
ONE = torch.tensor(1)
ZERO = torch.tensor(0)
INF = sys.maxsize
n_agents = 4
# Simulated data for n agents 
agent_bid_data = torch.FloatTensor(15000,n_agents).uniform_(0, 1)
test_bid_data = torch.FloatTensor(10000,n_agents).uniform_(0, 1)
agent_bid_data

tensor([[0.2432, 0.1826, 0.0171, 0.4254],
        [0.0922, 0.7191, 0.3426, 0.8171],
        [0.7166, 0.6420, 0.8883, 0.5163],
        ...,
        [0.8808, 0.7225, 0.3005, 0.0756],
        [0.7685, 0.1517, 0.9971, 0.7367],
        [0.7260, 0.5952, 0.1705, 0.4600]])

In [12]:
def input_formatter(data):
  # Input data has to be in the form of pairs of other peoples bids (for each person)
  data_dim = n_agents*(n_agents-1)
  formatted_data = torch.zeros(data_dim)

  for i, bid_set in enumerate(data):
    input_vector = torch.tensor([])
    for j, bid in enumerate(bid_set):
      input_vector = torch.cat((input_vector,bid_set[:j], bid_set[j+1:]), 0)
    # print(input_vector.shape)
    formatted_data = torch.vstack((formatted_data,input_vector))

  return formatted_data[1:]

In [13]:
input_data = input_formatter(agent_bid_data)
test_data = input_formatter(test_bid_data)
input_data_np = input_data.detach().cpu().numpy()
test_data_np = test_data.detach().cpu().numpy()

In [14]:
def vcg_constraints (values, rebates, verbose=False):
    num_agents = values.shape[0]    
    best_utility = max(values.sum(),1)
    project_utility = (num_agents)*best_utility
    achieved_utility = project_utility-rebates.sum()
    efficiency_ratio = achieved_utility/best_utility
    efficiency_constraint = rebates.sum() - ((num_agents - efficiency_ratio)*best_utility)
    budget_constraint = ((num_agents-1)*best_utility)-rebates.sum()
 
    if verbose:
      print('num_agents: ', num_agents)
      print('sum of bids: ', values.sum())
      print('sum of rebates: ', rebates.sum())
      print('best_utility: ', best_utility)
      print('project_utility: ', project_utility)
      print('achieved_utility: ', achieved_utility)
      print('efficiency_ratio: ', efficiency_ratio)
      print('budget_constraint: ', budget_constraint)
 
    return budget_constraint, efficiency_constraint, efficiency_ratio

In [15]:
uc = UpperBoundCalculator(n_agents)
upper_bound = uc.get_upper_bound()
print(f'Conjectured upper bound worst-case alpha for {n_agents} agents: {upper_bound}')

Conjectured upper bound worst-case alpha for 4 agents: 0.6666666666666667


In [16]:
def eval_single_genome(net, marker):
  max_budget_constraint = -INF
  max_efficiency_constraint = -INF
  worst_case_efficiency = INF
  for project in input_data_np:
    X = project
    output = net.activate(X[:marker])
    for i in range(marker):
      start = marker*(i+1)
      end = marker*(i+2)
      output_ = net.activate(X[start:end])
      output = np.concatenate((output, output_))
    budget_constraint, efficiency_constraint, efficiency_ratio = vcg_constraints(X[:n_agents], output)
    max_budget_constraint = max(max_budget_constraint, budget_constraint)
    max_efficiency_constraint = max(max_efficiency_constraint, efficiency_constraint)
    worst_case_efficiency =  min(worst_case_efficiency, efficiency_ratio)

  return max_budget_constraint, max_efficiency_constraint, worst_case_efficiency

def wc_efficiency_reward_function(wc_efficiency, ub):
    if wc_efficiency<0:
        return wc_efficiency*10
    else:
        return min(wc_efficiency, ub)

In [21]:
def eval_genomes_vcg(genomes, config):
    marker = n_agents-1
    budget_surplus_allowance = 0.1
    efficiency_surplus_allowance = 0.1
    constraint_surplus = budget_surplus_allowance + budget_surplus_allowance
    best_wc_budget_constraint = INF
    best_wc_efficiency_constraint = INF
    best_wc_efficiency = -INF
    best_fitness = -INF
    best_genome=None

    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        max_budget_constraint, max_efficiency_constraint, worst_case_efficiency = eval_single_genome(net, marker)
        penalty1 = 100*max_budget_constraint if max_budget_constraint>0 else (1e-2)*max_budget_constraint
        penalty2 = 10*max_efficiency_constraint if max_efficiency_constraint>0 else 0  
        genome.fitness = wc_efficiency_reward_function(worst_case_efficiency, upper_bound) - penalty1 - penalty2
        if genome.fitness>best_fitness:
            best_fitness = genome.fitness
            best_genome = genome
            best_wc_budget_constraint = max_budget_constraint
            best_wc_efficiency_constraint = max_efficiency_constraint
            best_wc_efficiency = worst_case_efficiency
    
    print('Best budget constraint: {0:3.5f}'.format(best_wc_budget_constraint))    
    print('Best efficiency constraint: {0:3.5f}'.format(best_wc_efficiency_constraint))    
    print('Best worst-case efficiency: {0:3.5f}'.format(best_wc_efficiency))


def lelu_activation(z):
        leaky = -0.0005
        return z if z > 0.0 else leaky * z    

def run_vcg(config_file, n_agents, stats=None, restore_mode=False, restored_population=None):
    marker = n_agents-1
    # Load configuration.
    
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    config.genome_config.activation_defs.add('lelu', lelu_activation)
    # Create the population, which is the top-level object for a NEAT run.
    if (restore_mode):
        p = restored_population
    else:
        p = Population(config)

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(StdOutReporter(True))
    if (restore_mode):
        p.add_reporter(stats)
    else:
        stats=StatisticsReporter(generation_interval=20, filename_prefix='neat-model2-stats-checkpoint-')
        p.add_reporter(stats)
    p.add_reporter(Checkpointer(generation_interval=20, time_interval_seconds=None, filename_prefix='neat-model2-checkpoint-'))

    
    # Run for specified number of generations.
    # winner = p.run(eval_genomes_vcg, 1)

    # Display the winning genome.
    # print('\nBest genome:\n{!s}'.format(winner))

    # for n, genome in enumerate(stats.most_fit_genomes):
    #     print(f'Generation {n}:{genome.fitness}')

    return p


In [22]:
# Loading config file
drive.mount('/content/gdrive', force_remount=True)
GDRIVE_DIR = "gdrive/My Drive/2. Adelaide Uni/Research Project/Evolutionary Models/Imports"
config_filename = "public-project-config-two.py"
session_config_filename = os.path.join(GDRIVE_DIR, config_filename)

if (reload_mode):
    population = run_vcg(session_config_filename, n_agents, stats_reporter, restore_mode=True, restored_population=restored_population)
else:
    population = run_vcg(session_config_filename, n_agents)
 


Mounted at /content/gdrive


In [ ]:
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         session_config_filename)


In [83]:
print(config.genome_config.input_keys)

[-1, -2, -3]


In [78]:
# for v in list():
def lelu_activation(z):
    leaky = -0.0005
    return z if z > 0.0 else leaky * z

config.genome_config.activation_defs.add('lelu', lelu_activation)
print(config.genome_config.activation_defs.get('lelu'))

<function lelu_activation at 0x7fbccc6d7b00>


In [81]:


for _,genome in population.population.items():
    print(genome)
    # net = neat.nn.FeedForwardNetwork.create(genome, config)
    # print(net)
    # for _, node in genome.nodes.items():
    #     print(node.bias)

    # print("\n")
    for _, connection in genome.connections.items():
        print(connection.key)
    break


Key: 1
Fitness: None
Nodes:
	0 DefaultNodeGene(key=0, bias=0.19877147085288654, response=1.0, activation=relu, aggregation=sum)
	1 DefaultNodeGene(key=1, bias=-2.0750161276072223, response=1.0, activation=relu, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-3, 1), weight=-0.014918516086590795, enabled=True)
	DefaultConnectionGene(key=(-2, 1), weight=0.013143505025953378, enabled=True)
	DefaultConnectionGene(key=(-1, 1), weight=0.0031722029281721874, enabled=True)
	DefaultConnectionGene(key=(1, 0), weight=0.008166621821381918, enabled=True)
(-1, 1)
(-2, 1)
(-3, 1)
(1, 0)


In [ ]:
# best_genome = evolution_stats.best_genome()
# marker  = n_agents-1
# config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
#                          neat.DefaultSpeciesSet, neat.DefaultStagnation,
#                          session_config_filename)
# best_net = neat.nn.FeedForwardNetwork.create(best_genome, config)
# max_budget_constraint, max_efficiency_constraint, worst_case_efficiency = eval_single_genome(best_net, marker)
# penalty1 = 100*max_budget_constraint if max_budget_constraint>0 else 0 
# penalty2 = 100*max_efficiency_constraint if max_efficiency_constraint>0 else 0  
# best_fitness = min(worst_case_efficiency, upper_bound) - penalty1 - penalty2


In [ ]:
# print('max_budget_constraint: ', max_budget_constraint)
# print('max_efficiency_constraint: ', max_efficiency_constraint)
# print('worst_case_efficiency: ', worst_case_efficiency)
# print('best_fitness: ', best_fitness)